In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import os

import re
import requests
import datetime
from datetime import datetime
from bs4 import BeautifulSoup
import bs4.element
from tqdm.notebook import tqdm
from konlpy.tag import Komoran

from sklearn.manifold import TSNE
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')

# Crawling 

In [12]:
def clean_html(x):
  x = re.sub("\&\w*\;","",x)
  x = re.sub("<.*?>","",x)
  return x


search_category = ['정치', '경제', '사회', '생활/문화', '세계', 'IT/과학']

client_id = "cByyPKiwDvI4YNRsAA4g" #1.에서 취득한 아이디 넣기
client_secret = "9v0xRpUYvj"  #1. 에서 취득한 키 넣기

search_word = 'IT' #검색어
encode_type = 'json' #출력 방식 json 또는 xml
max_display = 100 #출력 뉴스 수
sort = 'date' #결과값의 정렬기준 시간순 date, 관련도 순 sim
start = 1 # 출력 위치

url = f"https://openapi.naver.com/v1/search/news.{encode_type}?query={search_word}&display={str(int(max_display))}&start={str(int(start))}&sort={sort}"

#헤더에 아이디와 키 정보 넣기
headers = {'X-Naver-Client-Id' : client_id,
           'X-Naver-Client-Secret':client_secret
           }

r = requests.get(url, headers=headers)
#요청 결과 보기 200 이면 정상적으로 요청 완료
print(r)

r.json()

df = pd.DataFrame(r.json()['items'])
df['title'] = df['title'].apply(lambda x: clean_html(x))
df['description'] = df['description'].apply(lambda x: clean_html(x))

df.to_csv(f'news_search_result_{search_word}.csv', encoding = "utf-8-sig")

<Response [200]>


In [8]:
date = str(datetime.now())
date = date[:date.rfind(':')].replace(' ', '_')
date = date.replace(':','시') + '분'

def get_soup_obj(url):
    headers = { "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36" }
    res = requests.get(url, headers = headers)
    soup = BeautifulSoup(res.text,'lxml')
    
    return soup

def get_news_contents(url):
    soup = get_soup_obj(url)
    body = soup.find('div', class_="_article_body_contents")

    news_contents = ''
    for content in body:
        if type(content) is bs4.element.NavigableString and len(content) > 50:
            news_contents += content.strip() + ' '

    return news_contents

def get_news_info(url, s) : 
    default_img = "https://search.naver.com/search.naver?where=image&sm=tab_jum&query=naver#"
    current_page = 1     
    news_info_list = []

    for i in range (10) : 
        sec_url = url + s + "&date=" + "20211114&page=" + str(current_page)
        soup = get_soup_obj(sec_url)
        lis = soup.find('ul', class_='type06_headline').find_all("li", limit=10)
        
        for li in lis : 
            news_info = {
            "title" : li.img.attrs.get('alt') if li.img else li.a.text.replace("\n", "").replace("\t","").replace("\r","") , 
            "date" : li.find(class_="date").text,
            "news_url" : li.a.attrs.get('href', "nonetype"),
            "image_url" :  li.img.attrs.get('src') if li.img else default_img,
            "category" : s }
            print(news_info['news_url'])
            
            news_contents = get_news_contents(news_info['news_url'])
            news_info['contetns'] = news_contents
            news_info_list.append(news_info)
        
        current_page += 1 
    
    print(s + " 분야 크롤링 완료")    
    return news_info_list


sid = ['102', '103', '104', '105']
default_url = "https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1="
df = pd.DataFrame()

for s in tqdm(sid) : 
    news = get_news_info(default_url, s)
    df = df.append(news)

df.to_csv('네이버뉴스_{}.csv'.format(date), encoding = "utf-8-sig")

section url :  https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1=100
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=100&oid=366&aid=0000773798

https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=100&oid=119&aid=0002547905

https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=100&oid=087&aid=0000868320

https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=100&oid=087&aid=0000868248

https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=100&oid=087&aid=0000868242

https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=100&oid=087&aid=0000868241

https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=100&oid=087&aid=0000868239

https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=100&oid=005&aid=0001484190

https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=100&oid=005&aid=0001484189

https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=100&oid=277&aid=0004999274

section url :  https://news.nave

In [43]:
date = str(datetime.now())
date = date[:date.rfind(':')].replace(' ', '_')
date = date.replace(':','시') + '분'

def get_soup_obj(url):
    headers = { "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36" }
    res = requests.get(url, headers = headers)
    soup = BeautifulSoup(res.text,'lxml')
    
    return soup

def get_news_contents(url):
    soup = get_soup_obj(url)
    body = soup.select_one('div#articleBodyContents').text.strip()
    return body

def get_news_info(url, s) : 
    default_img = "https://search.naver.com/search.naver?where=image&sm=tab_jum&query=naver#"
    current_page = 1     
    news_info_list = []

    for i in range (10) : 
        sec_url = url + s + "&date=" + "20211114&page=" + str(current_page)
        soup = get_soup_obj(sec_url)
        lis = soup.find('ul', class_='type06_headline').find_all("li", limit=10)
        
        for li in lis : 
            news_info = {
            "title" : li.img.attrs.get('alt') if li.img else li.a.text.replace("\n", "").replace("\t","").replace("\r","") , 
            "date" : li.find(class_="date").text,
            "news_url" : li.a.attrs.get('href'),
            "image_url" :  li.img.attrs.get('src') if li.img else default_img,
            "category" : s }
            print(news_info['news_url'])
            print("fin")
            news_contents = get_news_contents(news_info['news_url'])
            news_info['contetns'] = news_contents
            news_info_list.append(news_info)
        
        current_page += 1 
    
    print(s + " 분야 크롤링 완료")    
    return news_info_list


sid = ['102', '103', '104', '105']
default_url = "https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1="
df = pd.DataFrame()

for s in tqdm(sid) : 
    news = get_news_info(default_url, s)
    df = df.append(news)

df.to_csv('네이버뉴스_{}.csv'.format(date), encoding = "utf-8-sig")

  0%|          | 0/4 [00:00<?, ?it/s]

https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=102&oid=081&aid=0003229859
fin
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=102&oid=214&aid=0001159896
fin
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=102&oid=005&aid=0001484184
fin
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=102&oid=214&aid=0001159894
fin
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=102&oid=214&aid=0001159893
fin
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=102&oid=001&aid=0012792203
fin
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=102&oid=001&aid=0012792202
fin
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=102&oid=001&aid=0012792201
fin
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=102&oid=023&aid=0003652966
fin
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=102&oid=469&aid=0000641116
fin
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=102&oid=032&aid=000

AttributeError: 'NoneType' object has no attribute 'text'